# Создание и заполнение данных БД Postgre

In [8]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor


In [9]:
POSTGRESQL_HOST = '10.129.0.25'

In [10]:
!env | grep POST

POSTGRESQL_USER=reader
POSTGRESQL_PASSWORD=Miba2021


In [11]:
!psql -c 'create database test;'

ERROR:  database "test" already exists


In [12]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
%%sql postgresql:///test
    SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

0 rows affected.


table


## Примеры SQL запросов 
1. Представим запрос для создания таблицы users c двумя полями:
```
%%sql postgresql:///test
    CREATE TABLE users (
        login varchar(10) NOT NULL, 
        password varchar(100)
    );
```

2. Чтобы добавить запись в таблицу, небходимо выполнить следующий запрос:

```
%%sql postgresql:///test
   INSERT INTO users VALUES ('monit', 'querty')
```

3. Получить данные из таблицы:
```
%%sql postgresql:///test
   SELECT * FROM users
```

4. Удалить таблицу:
```
!psql -d test -c 'DROP TABLE users'
```

Считаем данные из файла datasets/aggrigation_logs_per_week.csv.

Выведем первую строку с заголовками. 

In [14]:
import csv

with open('datasets/aggrigation_logs_per_week.csv') as file:
    reader = csv.reader(file)
    headers = next(reader)
print (headers)

['courseid', 'userid', 'num_week', 's_all', 's_all_avg', 's_course_viewed', 's_course_viewed_avg', 's_q_attempt_viewed', 's_q_attempt_viewed_avg', 's_a_course_module_viewed', 's_a_course_module_viewed_avg', 's_a_submission_status_viewed', 's_a_submission_status_viewed_avg', 'NameR_Level', 'Name_vAtt', 'Depart', 'Name_OsnO', 'Name_FormOPril', 'LevelEd', 'Num_Sem', 'Kurs', 'Date_vAtt']


Вам предоставлен набор данных с логами (действиями) студентов на образовательном портале за весенний семестр (агрегация по каждой неделе) по отдельному электронному курсу.
- сourseid — уникальный идентификатор курса, дисциплины;
- userid — уникальный идентификатор студента (не используется в обучении);
- num_week — номер недели в году;
- s_all — количество всех событий на текущий момент;
- s_all_avg — среднее количество всех событий в неделю;
- s_course_viewed — количество просмотров курса;
- s_course_viewed_avg — среднее количество просмотров курса в неделю;
- s_q_attempt_viewed — количество просмотров теста;
- s_q_attempt_viewed_avg — среднее количество просмотров теста в неделю;
- s_a_course_module_viewed — количество просмотров модуля в курсе;
- s_a_course_module_viewed_avg — среднее количество просмотров модуля в курсе в неделю;
- s_a_submission_status_viewed — количество отправленных заданий на проверку;
- s_a_submission_status_viewed_avg — среднее количество ответов;
- namer_level — оценка за дисциплину;
- depart — номер кафедры;
- name_osno — основа обучения (имеет два значения: бюджет или контракт);
- name_formopril — форма обучения;
- leveled — уровень образования (имеет два значения: бакалавриат, магистратура);
- num_sem — номер семестра;
- kurs — номер курса учебной группы.

Создадим таблицу с логами студентов в тестовой БД

In [15]:
!psql -d test -c 'CREATE TABLE logs ( \
    courseid varchar(5), \
    userid varchar(5), \
    num_week smallint, \
    s_all smallint, \
    s_all_avg varchar(10), \
    s_course_viewed smallint, \
    s_course_viewed_avg varchar(10), \
    s_q_attempt_viewed smallint, \
    s_q_attempt_viewed_avg varchar(10), \
    s_a_course_module_viewed smallint, \
    s_a_course_module_viewed_avg varchar(10), \
    s_a_submission_status_viewed smallint, \
    s_a_submission_status_viewed_avg varchar(10), \
    NameR_Level varchar(20), \
    Name_vAtt varchar(20), \
    Depart varchar(5), \
    Name_OsnO smallint, \
    Name_FormOPril smallint, \
    LevelEd smallint, \
    Num_Sem smallint, \
    Kurs smallint, \
    Date_vAtt varchar(20)\
);'

CREATE TABLE


Добавим данные из файла aggrigation_logs_per_week.csv в созданную таблицу.


In [16]:
!psql -d test \
    -c "\copy logs FROM 'datasets/aggrigation_logs_per_week.csv' DELIMITER ',' CSV HEADER;"

COPY 414528


Выведем первые 10 записей.

In [17]:
%%sql postgresql:///test
    SELECT * FROM logs LIMIT 10

10 rows affected.


courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,s_a_course_module_viewed_avg,s_a_submission_status_viewed,s_a_submission_status_viewed_avg,namer_level,name_vatt,depart,name_osno,name_formopril,leveled,num_sem,kurs,date_vatt
71262,34527,6,9,9,4,4,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,7,0,"4,5",0,2,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,8,0,3,0,"1,3333",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,9,0,"2,25",0,1,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,10,0,"1,8",0,"0,8",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,11,1,"1,6667",1,"0,8333",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,12,0,"1,4286",0,"0,7143",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,13,0,"1,25",0,"0,625",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,14,0,"1,1111",0,"0,5556",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,15,1,"1,1",1,"0,6",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022


## Задание 1:

Измените данные вещественного типа, сейчас целая и дробная часть разделены запятой, замените ее на точку. 

Выведите первые 10 записей, чтобы проверить результат предобработки. 

In [18]:
%%sql postgresql:///test

UPDATE logs 

SET 
    s_all_avg = REPLACE(s_all_avg, ',', '.'),
    s_course_viewed_avg = REPLACE(s_course_viewed_avg, ',', '.'),
    s_q_attempt_viewed_avg = REPLACE(s_q_attempt_viewed_avg, ',', '.'),
    s_a_course_module_viewed_avg = REPLACE(s_a_course_module_viewed_avg, ',', '.'),
    s_a_submission_status_viewed_avg = REPLACE(s_a_submission_status_viewed_avg, ',', '.');

SELECT * FROM logs LIMIT 10;

414528 rows affected.
10 rows affected.


courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,s_a_course_module_viewed_avg,s_a_submission_status_viewed,s_a_submission_status_viewed_avg,namer_level,name_vatt,depart,name_osno,name_formopril,leveled,num_sem,kurs,date_vatt
71269,7003,6,0,0,0,0,0,0,0,0,0,0,2,Экзамен,22,2,1,1,4,3,23.06.2022
71279,34612,6,0,0,0,0,0,0,0,0,0,0,2,Экзамен,22,1,1,1,2,2,22.06.2022
71349,35931,6,0,0,0,0,0,0,0,0,0,0,3,Экзамен,35,1,1,1,2,2,22.06.2022
71352,29333,21,68,13.625,9,2.875,0,0,24,3.3125,13,2,5,Экзамен,1,1,1,1,4,3,15.06.2022
71357,33422,11,4,20,1,5.5,0,0,0,2,0,1.6667,5,Экзамен,1,1,1,1,2,2,24.06.2022
71357,33429,6,16,16,5,5,0,0,0,0,0,0,5,Экзамен,1,1,1,1,2,2,24.06.2022
71357,33437,6,0,0,0,0,0,0,0,0,0,0,5,Экзамен,1,1,1,1,2,2,24.06.2022
71402,13700,13,0,7.25,0,2.125,0,0,0,1.125,0,0.875,2,Экзамен,1,1,1,1,6,4,24.06.2022
71402,24497,6,37,37,11,11,0,0,0,0,0,0,5,Экзамен,1,1,1,1,6,4,24.06.2022
71409,21973,25,0,13.25,0,2.15,0,0,0,2.8,0,1.95,5,Экзамен,1,1,1,1,8,5,20.06.2022


## Задание 2: 

Выведите количество кафедр, за которыми закреплены курсы на портале.





In [19]:
%%sql postgresql:///test
SELECT COUNT(DISTINCT depart) FROM logs;

1 rows affected.


count
43


##  Задание 3:

Выведите сколько у каждой кафедры закреплено электронных курсов на портале. 

У какой кафедры больше всего курсов на портале?

In [20]:
%%sql postgresql:///test
SELECT depart, COUNT(DISTINCT courseid) AS course_count
FROM logs
GROUP BY depart
ORDER BY course_count DESC;

43 rows affected.


depart,course_count
11,53
19,42
23,42
8,40
15,36
4,35
9,35
21,33
14,33
3,33


## Задание 4:

Имеется информация о кодировании каждой кафедры:

| Id  | Name     | Id  | Name     | Id  | Name     | Id  | Name     |
|-----|---------|-----|---------|-----|---------|-----|---------|
| 1   | АиИИ    | 12  | ИиИБ    | 23  | ПОиД    | 34  | УСиБА    |
| 2   | АСУ     | 13  | ИТМ     | 24  | Психол. | 35  | Физики   |
| 3   | АЭПиМ   | 14  | ЛиП     | 25  | ПЭиБЖД  | 36  | Физкульт.|
| 4   | БИиИТ   | 15  | ЛиУТС   | 26  | РМПИ    | 37  | Химии    |
| 5   | ВИ      | 16  | ЛПиМ    | 27  | РЯОЯиМК | 38  | ХОМ      |
| 6   | ВТиП    | 17  | Менеджм.| 28  | СРиППО  | 39  | ЦДОМ     |
| 7   | ГМДиОПИ | 18  | МиТОДиМ | 29  | CC      | 40  | ЭиМЭ     |
| 8   | ГМиТТК  | 19  | МиХТ    | 30  | ТиЭС    | 41  | Эконом.  |
| 9   | ГМУиУП  | 20  | ПиСЗ    | 31  | ТОМ     | 42  | ЭПП      |
| 10  | Дизайна | 21  | ПиЭММО  | 32  | ТССА    | 43  | ЯиЛ      |
| 11  | ДиСО    | 22  | ПМиИ    | 33  | УиИС    |     |          |


Создайте отдельно таблицу со справочником (номер кафедры и название(аббревиатура) кафедры) departments.

Внесите данные в таблицу. 

In [21]:
%%sql postgresql:///test

CREATE TABLE departments (
    id int PRIMARY KEY,
    name varchar(20)
);

Done.


[]

In [22]:
%%sql postgresql:///test

INSERT INTO departments VALUES
(1, 'АиИИ'), (2, 'АСУ'), (3, 'АЭПиМ'), (4, 'БИиИТ'), (5, 'ВИ'), 
(6, 'ВТиП'), (7, 'ГМДиОПИ'), (8, 'ГМиТТК'), (9, 'ГМУиУП'), (10, 'Дизайна'),
(11, 'ДиСО'), (12, 'ИиИБ'), (13, 'ИТМ'), (14, 'ЛиП'), (15, 'ЛиУТС'),
(16, 'ЛПиМ'), (17, 'Менеджм.'), (18, 'МиТОДиМ'), (19, 'МиХТ'), (20, 'ПиСЗ'),
(21, 'ПиЭММО'), (22, 'ПМиИ'), (23, 'ПОиД'), (24, 'Психол.'), (25, 'ПЭиБЖД'),
(26, 'РМПИ'), (27, 'РЯОЯиМК'), (28, 'СРиППО'), (29, 'CC'), (30, 'ТиЭС'),
(31, 'ТОМ'), (32, 'ТССА'), (33, 'УиИС'), (34, 'УСиБА'), (35, 'Физики'),
(36, 'Физкульт.'), (37, 'Химии'), (38, 'ХОМ'), (39, 'ЦДОМ'), (40, 'ЭиМЭ'),
(41, 'Эконом.'), (42, 'ЭПП'), (43, 'ЯиЛ');

43 rows affected.


[]

## Задание 5:

Выполните задание 3, но теперь нужно вывести не код кафедры, а ее название и количество курсов, которые за ней закреплены. 

У какой кафедры больше всего курсов?

In [23]:
%%sql postgresql:///test
SELECT departments.name, COUNT(DISTINCT courseid) AS course_count
FROM logs
JOIN departments ON logs.depart = departments.id::varchar
GROUP BY departments.name
ORDER BY course_count DESC
LIMIT 1;

1 rows affected.


name,course_count
ДиСО,53


## Задание 6:

Ответьте на вопрос: существуют ли курсы, за которыми закреплено несколько кафедр? Если такие курсы есть, то выведите их количество.
Также выведите названия кафедр, которые совместно преподают один и тот же курс.




In [24]:
%%sql postgresql:///test

SELECT COUNT(*)
FROM (
    SELECT courseid
    FROM logs
    GROUP BY courseid
    HAVING COUNT(DISTINCT depart) > 1
);

1 rows affected.


count
60


In [25]:
%%sql postgresql:///test

WITH multi_course AS (
    SELECT courseid
    FROM logs
    GROUP BY courseid
    HAVING COUNT(DISTINCT depart) > 1
)
SELECT DISTINCT logs.courseid, departments.name
FROM logs
JOIN departments ON logs.depart = departments.id::varchar
JOIN multi_course ON logs.courseid = multi_course.courseid
ORDER BY logs.courseid 
LIMIT 10;

10 rows affected.


courseid,name
71495,ГМиТТК
71495,ПиСЗ
71495,УиИС
71508,ПиСЗ
71508,УиИС
71541,ПиСЗ
71541,УиИС
71547,ПиСЗ
71547,УиИС
71549,ГМиТТК


## Задание 7:

Выведите количество студентов, которые получили 2, 3, 4, 5.

Пример вывода:

| namer_level |	count |
|-----|------|
|2 |	4 |
|3 |	3435 |
|4 | 	4676765|
|5 | 232 |


In [26]:
%%sql postgresql:///test
SELECT namer_level, COUNT(DISTINCT userid)
FROM logs
GROUP BY namer_level;

4 rows affected.


namer_level,count
2,1069
3,1884
4,3243
5,3407


## Задание 6:

Выведите студента, который больше всех работает на портале (у него максимальное количество логов за вест период обучения).

In [27]:
%%sql postgresql:///test
SELECT userid, SUM(s_all) AS count_logs
FROM logs
GROUP BY userid
ORDER BY count_logs DESC
LIMIT 1;

1 rows affected.


userid,count_logs
28871,10300


## Задание 9:

Выведите по каждой недели среднее количество всех событий на портале.

In [28]:
%%sql postgresql:///test
SELECT num_week, AVG(s_all)
FROM logs
GROUP BY num_week;

24 rows affected.


num_week,avg
6,13.7966072255673923
7,9.6161417322834646
8,8.0285433070866142
9,9.3932955071792497
10,8.2085456229735989
11,10.0220009263547939
12,9.3817160722556739
13,10.0140111162575266
14,9.8601783232978231
15,10.3536938397406207


## Задание 10: 

Выведите название кафедры, у которой больше всего отличников.

Отдельно выведите название кафедры, у которой больше всего двоечников. 

In [29]:
%%sql postgresql:///test

WITH top_students AS (
    SELECT depart, COUNT(DISTINCT userid)
    FROM logs
    WHERE namer_level = '5'
    GROUP BY depart
    ORDER BY count DESC
    LIMIT 1
)
SELECT departments.name, top_students.count
FROM top_students
JOIN departments ON top_students.depart = departments.id::varchar;

1 rows affected.


name,count
ДиСО,310


In [30]:
%%sql postgresql:///test

WITH top_students AS (
    SELECT depart, COUNT(DISTINCT userid)
    FROM logs
    WHERE namer_level = '2'
    GROUP BY depart
    ORDER BY count DESC
    LIMIT 1
)
SELECT departments.name, top_students.count
FROM top_students
JOIN departments ON top_students.depart = departments.id::varchar;

1 rows affected.


name,count
Эконом.,72
